In [1]:
#!pip3 install imagecodecs
#!pip3 install numpy tensorflow matplotlib scipy
#!pip3 install pandas
#!pip3 install tifffile
#!pip3 install -U scikit-learn scipy matplotlib
#!pip3 install tensorflow-addons
#!pip3 install tensorflow_decision_forests --upgrade

In [2]:
import math
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from numba import cuda 

import os

import pandas as pd

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_decision_forests as tfdf
import tifffile
from keras.layers import Flatten, Dense, Dropout
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import concatenate


import scipy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

2023-03-18 04:56:25.002903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 04:56:25.900387: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-18 04:56:25.900413: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-18 04:56:28.243814: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
def reset_gpu():
    device = cuda.get_current_device()
    device.reset()

In [4]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
   # plt.plot(history.history['accuracy'], label='accuracy')
   # plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.ylim([0, 
            max(
                max(history.history['loss']),
                max(history.history['val_loss'])
            )])
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.legend()
    plt.grid(True)

In [5]:
tiles = pd.read_csv(r'Data/Tiles_expanded.csv')
tiles_expanded_rv = pd.read_csv(r'Data/Tiles_expanded_road_vector.csv').drop(['Unnamed: 0'],axis=1)
tiles_expanded_rv = pd.get_dummies(tiles_expanded_rv, columns = ['RTTYP'])

In [6]:
tiles_expanded_rv.head()

,Tile_ID,Long2,Lat2,Long1,Lat1,Mid_lat,Mid_long,Stop_Signs,Paving_historical,Paving_future,Bus_stop,Collisions_Future,Collisions_Historical,RTTYP_I,RTTYP_M,RTTYP_O,RTTYP_S,RTTYP_U
0,36,-122.514446,37.779636,-122.513306,37.778732,37.779184,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
1,37,-122.514446,37.778732,-122.513306,37.777829,37.778280,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
2,151,-122.513306,37.779636,-122.512166,37.778732,37.779184,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
3,152,-122.513306,37.778732,-122.512166,37.777829,37.778280,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0
4,153,-122.513306,37.777829,-122.512166,37.776925,37.777377,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0


In [7]:
tiles_expanded_rv['Collisions_Future_binary']=tiles_expanded_rv['Collisions_Future'].apply(lambda x: 1 if x > 0 else 0)
tiles_expanded_rv['Collisions_Historical_binary']=tiles_expanded_rv['Collisions_Historical'].apply(lambda x: 1 if x > 0 else 0)

In [8]:
len(tiles)

10471

# Normalize data

In [9]:
def normalize_col(df, col_name):

    norm = (df[col_name] - df[col_name].min()) / (df[col_name].max() - df[col_name].min())
    return norm

tiles_expanded_rv['Mid_lat_norm'] = normalize_col(tiles_expanded_rv,'Mid_lat')
tiles_expanded_rv['Mid_long_norm'] = normalize_col(tiles_expanded_rv,'Mid_long')


Split into training and test

In [10]:
tiles_expanded_rv.columns

Index(['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical', 'RTTYP_I', 'RTTYP_M',
       'RTTYP_O', 'RTTYP_S', 'RTTYP_U', 'Collisions_Future_binary',
       'Collisions_Historical_binary', 'Mid_lat_norm', 'Mid_long_norm'],
      dtype='object')

In [11]:
MODEL_TYPE='linear' # 'linear'

if MODEL_TYPE == 'linear':
    input_x = ['Tile_ID','Collisions_Historical', 'Mid_lat_norm','Mid_long_norm', 'Stop_Signs', 'Paving_historical', 'Bus_stop','RTTYP_I','RTTYP_M','RTTYP_O','RTTYP_S','RTTYP_U']
    input_y = tiles_expanded_rv['Collisions_Future']
elif MODEL_TYPE == 'logistic':
    input_x = ['Tile_ID','Collisions_Historical_binary', 'Mid_lat_norm','Mid_long_norm', 'Stop_Signs', 'Paving_historical', 'Bus_stop','RTTYP_I','RTTYP_M','RTTYP_O','RTTYP_S','RTTYP_U']
    input_y = ['Collisions_Future_binary']
    

x_train, x_test, y_train, y_test = train_test_split(
                                    tiles_expanded_rv[input_x], 
                                    input_y,
                                    random_state=104, 
                                    test_size=0.20, 
                                    shuffle=True)



In [12]:
# np.asarray(x_train[['Collisions_Historical', 'Mid_lat_norm','Mid_long_norm', 'Stop_Signs', 'Paving_historical', 'Bus_stop','RTTYP_I','RTTYP_M','RTTYP_O','RTTYP_S','RTTYP_U']]).astype('float32').shape

In [13]:
len(x_train)

8376

In [14]:
len(y_train)

8376

In [15]:
len(x_test)

2095

In [16]:
len(y_test)

2095

Model #1: Linear regression using Historical Collisions, SFMTA Street Characteristics, and Geocoordinates

## Image Pre-Processing

In [17]:
IMAGE_PATH = '../210_Capstone_Aditya_Arisa_Noriel/Satellite Imagery/Satellite Images Tiled/' 

In [18]:
#eda
img = tifffile.imread('../210_Capstone_Aditya_Arisa_Noriel/Satellite Imagery/Satellite Images Tiled/36.tif')[0:148, 0:188, :]
img2 = tifffile.imread('../210_Capstone_Aditya_Arisa_Noriel/Satellite Imagery/Satellite Images Tiled/37.tif')[0:148, 0:188, :]

In [19]:
# eda
print(np.shape(img2))

(148, 188, 4)


In [20]:
img

array([[[185., 177., 160., 130.],
        [184., 178., 161., 130.],
        [184., 178., 160., 130.],
        ...,
        [105., 108.,  70., 173.],
        [117., 117.,  82., 166.],
        [112., 126.,  76., 182.]],

       [[184., 178., 161., 131.],
        [184., 178., 160., 130.],
        [183., 177., 159., 129.],
        ...,
        [104., 108.,  71., 173.],
        [113., 115.,  80., 163.],
        [110., 123.,  76., 177.]],

       [[183., 178., 159., 128.],
        [183., 177., 159., 127.],
        [183., 177., 159., 127.],
        ...,
        [101., 110.,  71., 170.],
        [109., 116.,  77., 168.],
        [110., 124.,  78., 179.]],

       ...,

       [[141., 131., 103.,  53.],
        [142., 131., 102.,  56.],
        [148., 139., 108.,  62.],
        ...,
        [ 72.,  88.,  51., 101.],
        [ 78.,  92.,  54., 109.],
        [ 82.,  97.,  56., 116.]],

       [[138., 130., 100.,  52.],
        [140., 131., 101.,  54.],
        [146., 136., 107.,  60.],
        .

In [21]:
# eda
test = []
test.append(img[0:148, 0:188, :])
test.append(img2[0:148, 0:188, :])
print(np.shape(test))
test = np.stack(test)
print(np.shape(test))

(2, 148, 188, 4)
(2, 148, 188, 4)


In [22]:
#example of accessing RGB digits and storing average in a list

list_ = []

print(img[0][0])
print([np.mean(img[0][0][0:3]), img[0][0][3]])

for i in img:
    for j in i:
        list_.append([np.mean(j[0:3]), j[3]])

print(list_[0])

[185. 177. 160. 130.]
[174.0, 130.0]
[174.0, 130.0]


In [ ]:
def preprocess_data_part1(IMAGE_PATH):
    """ Generate lists of images and labelsbased on temp_no_refer and temp_refer lists
    
    Params:
    -------
    IMAGE_PATH (str): path to directory with images.
    
    Returns:
    --------
    images_mini  (np.ndarray): Images of shape (N, 149 3)
    """
    
    data_mini = []
    
    for id in x_train['Tile_ID']:    
                    
        # read image and store as matrix            
        # Index at the end makes all images the same size (they sometimes differ by 1 pixel)
        image = tifffile.imread(IMAGE_PATH + str(id) + '.tif')[0:148, 0:188, :]
        
        #image_grayscale = []

        #for i in image:
        #    for j in i:
        #        image_grayscale.append([np.mean(j[0:3]), j[3]])
            
        # append to images
        data_mini.append(np.mean(image))
 
    # stack images and trasnform to array
    images_mini = np.stack(data_mini)
    
    return images_mini

In [24]:
# THERE IS ONE FILE IN THE "SATELLITE IMAGES TILED" FOLDER THAT IS NOT A TIF IMAGE, DELETE THIS FILE
images_mini = preprocess_data_part1(IMAGE_PATH)

In [25]:
np.shape(images_mini)

(8376, 148, 188, 4)

In [26]:
# def preprocess_data_part2(image):
#     """ Generate lists of images and labelsbased on temp_no_refer and temp_refer lists
    
#     Params:
#     -------
#     IMAGE_PATH (str): path to directory with images.
    
#     Returns:
#     --------
#     images_mini  (np.ndarray): Images of shape (N, 149 3)
#     """
    
#     image_grayscale = []

#     for i in image:
#         for j in i:
#             image_grayscale.append([np.mean(j[0:3]), j[3]])
            
#     # stack images and trasnform to array
#     #images_mini = np.stack(image_grayscale)
    
#     return images_mini

In [27]:
# images_mini = preprocess_data_part2(images_mini)

In [28]:
# images_mini2

In [29]:
# images_mini_reshaped = images_mini.reshape((8376,1,1,111296))

In [30]:
# images_mini_reshaped.shape
# len(images_mini_reshaped[0][0][0])

Street Data Pre-Processing

In [31]:
street = np.asarray(x_train[input_x[1:]]).astype('float32')
street_mini = []
for row in street:
    street_mini.append([[row]])
street_mini = np.stack(street_mini)
np.shape(street_mini)

(8376, 1, 1, 11)

In [32]:
street_mini_reshaped = street_mini.reshape(8376,11)
np.shape(street_mini_reshaped)

(8376, 11)

In [33]:
#images_mini_reshaped = images_mini.reshape(8376,1)
#np.shape(images_mini_reshaped)

In [34]:
input_image_street = np.hstack(
    (street_mini.reshape((8376,11)),
     images_mini.reshape(8376,111296))
)

In [35]:
# input_image_street = input_image_street.reshape((8376,1,1, 111307))

In [36]:
input_image_street[0]

array([  2.        ,   0.40708277,   0.69343007, ..., 145.        ,
       106.        , 177.        ], dtype=float32)

In [37]:
input_image_street.shape
# y_train.shape

(8376, 111307)

In [38]:
np.shape(y_train)

(8376,)

**Model #3: Random Forest using Historical Collisions, SFMTA Street Characteristics, and Geocoordinates**

Helpful sources:

https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/RandomForestModel
https://www.tensorflow.org/decision_forests/tutorials/beginner_colab
https://www.youtube.com/watch?v=5qgk9QJ4rdQ&t=181s

In [ ]:
# Train a Random Forest model.

rf_model = tfdf.keras.RandomForestModel(task=2, num_trees=100)

rf_model.fit(input_image_street,
    y_train,
    #validation_data=[x_test[['Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop']], y_test],     
    epochs=1,
    verbose=1)

Use /tmp/tmp1u4loj_k as temporary training directory
Reading training dataset...


2023-03-18 04:57:38.233542: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-18 04:57:38.234473: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-18 04:57:38.234508: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-13-91): /proc/driver/nvidia/version does not exist
2023-03-18 04:57:38.236602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 04:57:38.344498: W tensorflow/tsl/framework/cpu_allocator

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
# Train a Random Forest model.

rf_model2 = tfdf.keras.RandomForestModel(task=2, num_trees=300)

rf_model2.fit(input_image_street,
    y_train,
    #validation_data=[x_test[['Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop']], y_test],     
    epochs=1,
    verbose=1)

In [ ]:
print(rf_model2.summary())

In [ ]:
##uncomment to show training log
#rf_model_train.make_inspector().training_logs()
rf_model2.make_inspector().evaluation()

In [ ]:
x_test.columns

In [ ]:
#evaluate the model
rf_model2.evaluate(x_test[['Tile_ID', 'Collisions_Historical', 'Mid_lat_norm', 'Mid_long_norm',
       'Stop_Signs', 'Paving_historical', 'Bus_stop', 'RTTYP_I', 'RTTYP_M',
       'RTTYP_O', 'RTTYP_S', 'RTTYP_U']])

In [ ]:
#make predictions from test set
pd.DataFrame(rf_model.predict(x_test[['Tile_ID', 'Collisions_Historical', 'Mid_lat_norm', 'Mid_long_norm',
       'Stop_Signs', 'Paving_historical', 'Bus_stop', 'RTTYP_I', 'RTTYP_M',
       'RTTYP_O', 'RTTYP_S', 'RTTYP_U']]))

In [ ]:
pd.DataFrame(rf_model.predict(x_test[['Tile_ID', 'Collisions_Historical', 'Mid_lat_norm', 'Mid_long_norm',
       'Stop_Signs', 'Paving_historical', 'Bus_stop', 'RTTYP_I', 'RTTYP_M',
       'RTTYP_O', 'RTTYP_S', 'RTTYP_U']])).hist()

In [ ]:
import matplotlib.pyplot as plt

logs = rf_model.make_inspector().training_logs()

plt.plot([log.num_trees for log in logs], [log.evaluation.rmse for log in logs], label="training data")
plt.xlabel("Number of trees")
plt.ylabel("RMSE (out-of-bag)")
plt.legend()
plt.show()

**Model #4: Gradient Boosted Tree using Historical Collisions, SFMTA Street Characteristics, and Geocoordinates**

Helpful sources:

https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/GradientBoostedTreesModel
https://www.tensorflow.org/decision_forests/tutorials/beginner_colab

In [ ]:
gbt_model = tfdf.keras.GradientBoostedTreesModel(task=2)

gbt_model.fit(input_image_street,
    y_train,
    #validation_data=[x_test[['Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop']], y_test],     
    epochs=1,
    verbose=1)

In [ ]:
print(gbt_model.summary())

In [ ]:
##uncomment to show training log
#gbt_model.make_inspector().training_logs()
gbt_model.make_inspector().evaluation()

In [ ]:
import matplotlib.pyplot as plt

logs = gbt_model.make_inspector().training_logs()

plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs], label="training data")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

logs = gbt_model.make_inspector().training_logs()

plt.plot([log.num_trees for log in logs], [log.evaluation.rmse for log in logs], label="training data")
plt.xlabel("Number of trees")
plt.ylabel("RMSE")
plt.legend()

plt.show()

In [ ]:
#evaluate the model
#gbt_model.evaluate(x_test[['Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop']])

In [ ]:
#make predictions from test set
#pd.DataFrame(gbt_model.predict(x_test[['Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop']]))

**Baseline Linear Model**

In [ ]:
# x: 'Collisions_Historical', 'Mid_lat','Mid_long', 'Stop_Signs', 'Paving_historical', 'Bus_stop','RTTYP_I','RTTYP_M','RTTYP_O','RTTYP_S','RTTYP_U'
# y: 'Collisions_Future'

def create_linear_model():
    # DENSE LAYERS 
    input = tf.keras.layers.Input(shape=(12,))

    outputs = layers.Dense(1)(input)

    linear_model = tf.keras.models.Model(inputs=input, outputs=outputs, name="linear_model")

    linear_model.summary()

    linear_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mse'
        )
    
    return linear_model

linear_model = create_linear_model()

history = linear_model.fit(input_image_street,
     y_train,
    epochs=100,
    # Suppress logging.
     verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
plot_loss(history)

In [ ]:
# def build_model1():
#     model = keras.Sequential([
#         layers.Dense(64, activation=tf.nn.relu, input_shape=(12,)),
#         layers.Dense(64, activation=tf.nn.relu),
#         layers.Dense(1)
#     ])
    
#     optimizer = tf.keras.optimizers.RMSprop(0.001)
    
#     model.compile(loss='mse',
#                   optimizer=optimizer,
#                   metrics=['mse'])
    
#     return model

In [ ]:
# model1 = build_model1()

In [ ]:
# model1.summary()

In [ ]:
# history1 = model1.fit(
#     input_image_street,
#      y_train,
#     epochs=100,
#     # Suppress logging.
#      verbose=1,
#     # Calculate validation results on 20% of the training data.
#     validation_split = 0.2)